In [1]:
import pandas as pd
import os
import sys
import yaml
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
week, weeks2, weeks3, weeks4, weeks5, weeks6 = 7, 14, 21, 28, 35, 42
weeks_list = [week, weeks2, weeks3, weeks4, weeks5, weeks6]
raw_data_path = 'raw/'
original_path = 'ourpped/'
new_data_path = 'preprocessed_rania_zeros/'

In [3]:
def add_gm_features(stock_sample): #provide ourpped path
    mult_percentage = 100
    rows_indices = stock_sample.iloc[:, 4] == -123321
    stock_sample.loc[rows_indices, 4] = 0
#     print(rows_indices)
#     print(stock_sample.loc[rows_indices, 4])
    stock_sample.loc[:,'gm_week'] = stock_sample.iloc[:, 4].rolling(week).mean()
    stock_sample.loc[:,'gm_2_weeks'] = stock_sample.iloc[:, 4].rolling(weeks2).mean()
    stock_sample.loc[:,'gm_3_weeks'] = stock_sample.iloc[:, 4].rolling(weeks3).mean()
    stock_sample.loc[:,'gm_4_weeks'] = stock_sample.iloc[:, 4].rolling(weeks4).mean()
    stock_sample.loc[:,'gm_5_weeks'] = stock_sample.iloc[:, 4].rolling(weeks5).mean()
    stock_sample.loc[:,'gm_6_weeks'] = stock_sample.iloc[:, 4].rolling(weeks6).mean()
    
    stock_sample.loc[:,'gm_week'] = mult_percentage*(
    stock_sample.loc[:,'gm_week']/stock_sample.iloc[:,4] -1)

    stock_sample.loc[:,'gm_2_weeks'] = mult_percentage*(
        stock_sample.loc[:,'gm_2_weeks']/stock_sample.iloc[:, 4] -1)

    stock_sample.loc[:,'gm_3_weeks'] = mult_percentage*(
        stock_sample.loc[:,'gm_3_weeks']/stock_sample.iloc[:, 4] -1)

    stock_sample.loc[:,'gm_4_weeks'] = mult_percentage*(
        stock_sample.loc[:,'gm_4_weeks']/stock_sample.iloc[:, 4] -1)

    stock_sample.loc[:,'gm_5_weeks'] = mult_percentage*(
        stock_sample.loc[:,'gm_5_weeks']/stock_sample.iloc[:, 4] -1)

    stock_sample.loc[:,'gm_6_weeks'] = mult_percentage*(
        stock_sample.loc[:,'gm_6_weeks']/stock_sample.iloc[:, 4] -1)

In [4]:
def add_pr_features(stock_sample): #provide ourpped path
    rows_indices = stock_sample.iloc[:, 4] == -123321
    stock_sample.loc[rows_indices, 4] = 0
    stock_sample.loc[:,'pr_week'] = stock_sample.iloc[:, 4].rolling(week).agg(lambda x: (x>0).mean())
    stock_sample.loc[:,'pr_2_weeks'] = stock_sample.iloc[:, 4].rolling(weeks2).agg(lambda x: (x>0).mean())
    stock_sample.loc[:,'pr_3_weeks'] = stock_sample.iloc[:,  4].rolling(weeks3).agg(lambda x: (x>0).mean())
    stock_sample.loc[:,'pr_4_weeks'] = stock_sample.iloc[:, 4].rolling(weeks4).agg(lambda x: (x>0).mean())
    stock_sample.loc[:,'pr_5_weeks'] = stock_sample.iloc[:, 4].rolling(weeks5).agg(lambda x: (x>0).mean())
    stock_sample.loc[:,'pr_6_weeks'] = stock_sample.iloc[:, 4].rolling(weeks6).agg(lambda x: (x>0).mean())

    stock_sample.loc[:,'pr_week'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_week'].values.reshape(-1,1))
    stock_sample.loc[:,'pr_2_weeks'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_2_weeks'].values.reshape(-1,1))
    stock_sample.loc[:,'pr_3_weeks'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_3_weeks'].values.reshape(-1,1))
    stock_sample.loc[:,'pr_4_weeks'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_4_weeks'].values.reshape(-1,1))
    stock_sample.loc[:,'pr_5_weeks'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_5_weeks'].values.reshape(-1,1))
    stock_sample.loc[:,'pr_6_weeks'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_6_weeks'].values.reshape(-1,1))

In [5]:
def add_features(stock):
#     stock_path = os.path.join(original_path, '{}'.format(stock))
#     stock_sample = pd.read_csv(stock_path)
#     add_gm_features(stock_sample)
#     add_pr_features(stock_sample)
    add_gm_features(stock)
    add_pr_features(stock)
    
#     df = stock_sample.copy()
    df = stock.copy()
    
    return np.vstack([df['gm_week'].values, df['gm_2_weeks'].values,
                      df['gm_3_weeks'].values, df['gm_4_weeks'].values,
                      df['gm_5_weeks'].values, df['gm_6_weeks'].values,
                      df['pr_week'].values, df['pr_2_weeks'].values,
                     df['pr_3_weeks'].values, df['pr_4_weeks'].values,
                     df['pr_5_weeks'].values, df['pr_6_weeks'].values]).T

In [6]:
def changeOurppedFiles(stock, num_rows=0):
    #original df
    stock_path = os.path.join(original_path, '{}'.format(stock))
    df_orig = pd.read_csv(stock_path,header=None)
    df_new = df_orig.copy()
    #insert new features 
    if num_rows == 0:
        tmp_last_columns = df_new.iloc[:,-2:].copy()
        df_new.drop(df_new.iloc[:, -2:], inplace=True, axis=1)
        newfeatures  = add_features(df_new)
#         newfeatures  = add_features(stock)     
#         df_new = pd.concat([df_new, pd.DataFrame(newfeatures)], axis=1)
        df_new = pd.concat([df_new, pd.DataFrame(tmp_last_columns)], axis=1)
    else:
        tmp_last_columns = df_new.iloc[-num_row:,-2:].copy()
        #TODO: continue
    return df_new

In [7]:
def fillMissingData(df):
    df.replace(' ', np.nan, inplace=True)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.mean(skipna=True, numeric_only=True), inplace=True)
    rows_indices = df.iloc[:, 1] == -123321
    df.loc[rows_indices, :] = -123321

In [8]:
experement_path = 'preprocessed_rania_experementing_data_fill/'
for stock in os.listdir(original_path):
    try:
        df_new = changeOurppedFiles(stock)
        #df_new = norm_df(df_new)
        fillMissingData(df_new)
        df_new.to_csv(os.path.join(experement_path,'{}'.format(stock)),header=None, index=None)
    except Exception as e:
        print(e)
        print('except:', stock)

## FILL IN MISSING CELLS

In [320]:
experement_path = 'preprocessed_rania_experementing_data_fill/'

In [321]:
stock_path = os.path.join(experement_path, '{}'.format('AMZN.csv'))
stock_file = pd.read_csv(stock_path,header=None)

In [322]:
stock_file.head(30)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-123321.0,-123321.000000
1,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-123321.0,-123321.000000
2,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-123321.0,-123321.000000
3,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-123321.0,-123321.000000
4,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-123321.0,-123321.000000
5,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-123321.0,-123321.000000
6,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,-123321.0,-123321.000000
7,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,-123321.0,-123321.000000
8,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,-123321.0,-123321.000000
9,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,-123321.0,-123321.000000


In [324]:
stock_file.replace(' ', np.nan, inplace=True)
stock_file.replace([np.inf, -np.inf], np.nan, inplace=True)

In [325]:
stock_file.fillna(stock_file.mean(skipna=True, numeric_only=True), inplace=True)

In [326]:
stock_file.iloc[20,:]

0    -123321.000000
1    -123321.000000
2    -123321.000000
3    -123321.000000
4    -123321.000000
5    -123321.000000
6    -123321.000000
7    -123321.000000
8    -123321.000000
9    -123321.000000
10   -123321.000000
11         0.000000
12         0.000000
13         0.000000
14     42134.559018
15     23674.899842
16     53258.352819
17         0.000000
18         0.000000
19         0.000000
20         0.709357
21         0.641423
22         0.612908
23   -123321.000000
24   -123321.000000
Name: 20, dtype: float64

In [327]:
rows_indices = stock_file.iloc[:, 1] == -123321
stock_file.loc[rows_indices, 11:22] = -123321

In [328]:
stock_file.to_csv(os.path.join(experement_path, 'AMZN_working.csv'), header=None, index=None)

In [5]:
stock_path = os.path.join(original_path, '{}'.format('XOM.csv'))
df_orig = pd.read_csv(stock_path,header=None)
rows_indices = df_orig.iloc[:, 1] == -123321
print(rows_indices)
print(df_orig.loc[rows_indices, 4])

0        True
1        True
2        True
3        True
4        True
        ...  
2513    False
2514    False
2515    False
2516    False
2517    False
Name: 1, Length: 2518, dtype: bool
0    -123321.0
1    -123321.0
2    -123321.0
3    -123321.0
4    -123321.0
5    -123321.0
6    -123321.0
7    -123321.0
8    -123321.0
9    -123321.0
10   -123321.0
11   -123321.0
12   -123321.0
13   -123321.0
14   -123321.0
15   -123321.0
16   -123321.0
17   -123321.0
18   -123321.0
19   -123321.0
20   -123321.0
21   -123321.0
22   -123321.0
23   -123321.0
24   -123321.0
25   -123321.0
26   -123321.0
27   -123321.0
28   -123321.0
Name: 4, dtype: float64


In [6]:
df_orig.loc[rows_indices, 4] = 0
print(df_orig.loc[rows_indices, 4])

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
Name: 4, dtype: float64


#### TESTING - 0 vs -123321

In [5]:
stock_path = os.path.join(original_path, '{}'.format('AAPL.csv'))
df_orig = pd.read_csv(stock_path,header=None)
stock_sample = df_orig.copy()
tmp_last_columns = stock_sample.iloc[:,-2:].copy()

In [6]:
mult_percentage = 100
rows_indices = stock_sample.iloc[:, 4] == -123321
print(stock_sample.head(5))
stock_sample.loc[rows_indices, 4] = 0
print(rows_indices)
print(stock_sample.head(5))
stock_sample.loc[:,'gm_week'] = stock_sample.iloc[:, 4].rolling(week).mean()
print(stock_sample.iloc[25:32, 4])
print(stock_sample.iloc[:, 4].rolling(week).mean()[25:32])
stock_sample.loc[:,'gm_2_weeks'] = stock_sample.iloc[:, 4].rolling(weeks2).mean()
stock_sample.loc[:,'gm_3_weeks'] = stock_sample.iloc[:, 4].rolling(weeks3).mean()
stock_sample.loc[:,'gm_4_weeks'] = stock_sample.iloc[:, 4].rolling(weeks4).mean()
stock_sample.loc[:,'gm_5_weeks'] = stock_sample.iloc[:, 4].rolling(weeks5).mean()
stock_sample.loc[:,'gm_6_weeks'] = stock_sample.iloc[:, 4].rolling(weeks6).mean()

stock_sample.loc[:,'gm_week'] = mult_percentage*(
stock_sample.loc[:,'gm_week']/stock_sample.iloc[:,4] -1)

stock_sample.loc[:,'gm_2_weeks'] = mult_percentage*(
    stock_sample.loc[:,'gm_2_weeks']/stock_sample.iloc[:, 4] -1)

stock_sample.loc[:,'gm_3_weeks'] = mult_percentage*(
    stock_sample.loc[:,'gm_3_weeks']/stock_sample.iloc[:, 4] -1)

stock_sample.loc[:,'gm_4_weeks'] = mult_percentage*(
    stock_sample.loc[:,'gm_4_weeks']/stock_sample.iloc[:, 4] -1)

stock_sample.loc[:,'gm_5_weeks'] = mult_percentage*(
    stock_sample.loc[:,'gm_5_weeks']/stock_sample.iloc[:, 4] -1)

stock_sample.loc[:,'gm_6_weeks'] = mult_percentage*(
    stock_sample.loc[:,'gm_6_weeks']/stock_sample.iloc[:, 4] -1)

         0         1         2         3         4         5         6   \
0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0   
1 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0   
2 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0   
3 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0   
4 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0   

         7         8         9         10        11        12  
0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0  
1 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0  
2 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0  
3 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0  
4 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0 -123321.0  
0        True
1        True
2        True
3        True
4        True
        ...  
2513    False
2514    False
2515    False
2516    False
2517    False
Name: 4, Le

In [7]:
stock_sample.loc[:,'pr_week'] = stock_sample.iloc[:, 4].rolling(week).agg(lambda x: (x>0).mean())
stock_sample.loc[:,'pr_2_weeks'] = stock_sample.iloc[:, 4].rolling(weeks2).agg(lambda x: (x>0).mean())
stock_sample.loc[:,'pr_3_weeks'] = stock_sample.iloc[:,  4].rolling(weeks3).agg(lambda x: (x>0).mean())
stock_sample.loc[:,'pr_4_weeks'] = stock_sample.iloc[:, 4].rolling(weeks4).agg(lambda x: (x>0).mean())
stock_sample.loc[:,'pr_5_weeks'] = stock_sample.iloc[:, 4].rolling(weeks5).agg(lambda x: (x>0).mean())
stock_sample.loc[:,'pr_6_weeks'] = stock_sample.iloc[:, 4].rolling(weeks6).agg(lambda x: (x>0).mean())

stock_sample.loc[:,'pr_week'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_week'].values.reshape(-1,1))
stock_sample.loc[:,'pr_2_weeks'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_2_weeks'].values.reshape(-1,1))
stock_sample.loc[:,'pr_3_weeks'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_3_weeks'].values.reshape(-1,1))
stock_sample.loc[:,'pr_4_weeks'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_4_weeks'].values.reshape(-1,1))
stock_sample.loc[:,'pr_5_weeks'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_5_weeks'].values.reshape(-1,1))
stock_sample.loc[:,'pr_6_weeks'] = MinMaxScaler().fit_transform(stock_sample.loc[:, 'pr_6_weeks'].values.reshape(-1,1))

In [8]:
df = stock_sample.copy()
new_features_cols = np.vstack([df['gm_week'].values, df['gm_2_weeks'].values,
                      df['gm_3_weeks'].values, df['gm_4_weeks'].values,
                      df['gm_5_weeks'].values, df['gm_6_weeks'].values,
                      df['pr_week'].values, df['pr_2_weeks'].values,
                     df['pr_3_weeks'].values, df['pr_4_weeks'].values,
                     df['pr_5_weeks'].values, df['pr_6_weeks'].values]).T

In [10]:
df.head(32)

,0,1,2,3,4,5,6,7,8,9,...,gm_3_weeks,gm_4_weeks,gm_5_weeks,gm_6_weeks,pr_week,pr_2_weeks,pr_3_weeks,pr_4_weeks,pr_5_weeks,pr_6_weeks
0,-123321.000000,-123321.000000,-123321.000000,-123321.000000,0.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-123321.000000,-123321.000000,-123321.000000,-123321.000000,0.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-123321.000000,-123321.000000,-123321.000000,-123321.000000,0.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-123321.000000,-123321.000000,-123321.000000,-123321.000000,0.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-123321.000000,-123321.000000,-123321.000000,-123321.000000,0.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-123321.000000,-123321.000000,-123321.000000,-123321.000000,0.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-123321.000000,-123321.000000,-123321.000000,-123321.000000,0.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
7,-123321.000000,-123321.000000,-123321.000000,-123321.000000,0.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
8,-123321.000000,-123321.000000,-123321.000000,-123321.000000,0.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
9,-123321.000000,-123321.000000,-123321.000000,-123321.000000,0.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,-123321.000000,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN


In [12]:
df_new = df.copy()
df_new.drop(df_new.iloc[:, -2:], inplace=True, axis=1)
df_new = pd.concat([df_new, pd.DataFrame(new_features_cols)], axis=1)
df_new = pd.concat([df_new, pd.DataFrame(tmp_last_columns)], axis=1)

In [13]:
df_new.to_csv(os.path.join(new_data_path,'{}'.format('AAPL.csv')),header=None, index=None)